In [1]:
cd C:\Users\Public\yolov71

C:\Users\Public\yolov71


### Fonction d'extraction de l'ouput du model 


In [3]:
def fun(w):
    output_string=w[15]
    index = output_string.find(',')
    class_string = output_string[2:index]
    return class_string



### Fonction pour comparer le resultat de l'ocr avec les marques de bouteilles et choisir la marque la plus proche en terme grammatical de l'output



In [3]:
import Levenshtein

words = ['FANTA','fanta', 'cocacola', 'COCACOLA', 'BOGA', 'apla', 'SCHWEPPES' ,'sabrine', 'schweppes', 'boga', 'APLA', 'SPRITE','orangina', 'ORANGINA', 'sprite','SABRINE','SAFIA','FOURAT','MELLITI', 'delice','Tijen','Marwa', 'Janet', 'Dima']

def find_most_similar_words(word, words_list):
    similar_words = []
    distances = []
    for w in words_list:
        distance = Levenshtein.distance(word, w)
        distances.append(distance)
        if distance == min(distances):
            similar_words.append(w)
    sorted_words = sorted(zip(similar_words, distances), key=lambda x: x[1])
    return [w for w, _ in sorted_words]


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


### Ce code travaille en boucle tant que une personne a fait un login , ce code detectera la bouteille , appliquera les models et enregistre les resultat dans un sheet

In [42]:
import time
import os
import cv2
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import torch
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import easyocr
from pylab import rcParams
from IPython.display import Image
rcParams['figure.figsize'] = 8, 16
import openpyxl

class MyHandler(FileSystemEventHandler):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def on_created(self, event):
        if event.is_directory:
            return
        if event.src_path.endswith('.jpg'):
            image = cv2.imread(event.src_path)
            # Apply the machine learning model to the image
            x=!python detect.py --source C:\Users\Public\yolov71\runs\detect\exp5\test1.jpg --weights C:\Users\Public\yolov71\runs\train\exp12\weights\best.pt
            y=!python detect.py --source C:\Users\Public\yolov71\runs\detect\exp5\test1.jpg --weights C:\Users\Public\yolov71\runs\train\exp12\weights\defective.pt        
            z=!python cap.py --source C:\Users\Public\yolov71\runs\detect\exp5\test1.jpg --weights C:\Users\Public\yolov71\runs\train\exp12\weights\cap.pt    
          
            reader = easyocr.Reader(["en"])
            output4 = reader.readtext('C:/Users/Public/yolov71/runs/detect/exp5/test.jpg')
            text_output4 = ""
            for result in output4:
                text_output4 += result[1]

            print(text_output4)
           
                                                     
            # Load the Excel file
            workbook = openpyxl.load_workbook('data.xlsx')

            # Select a specific sheet in the workbook
            sheet = workbook['Sheet1']

            last_row = sheet.max_row

            # Write data to the next row in the sheet
            sheet.cell(row=last_row+1, column=1, value=x)
            sheet.cell(row=last_row+1, column=2, value=y)
            sheet.cell(row=last_row+1, column=3, value=z)
            sheet.cell(row=last_row+1, column=4, value=text_output4)
            

            workbook.save('data.xlsx')

            print("Data written to Excel file successfully.")
#Detection s'il y a une image inserée dans le dossier partagé , il va appliquer les modeles sur l'image
#puis enregistrer les resultats dans un sheet
if __name__ == "__main__":
    # Load the machine learning model here
    model = !python detect.py --source C:\Users\Public\yolov71\runs\detect\exp5\test.jpg --weights C:\Users\Public\yolov71\runs\train\exp12\weights\best.pt
    path = "C:/Users/Public/yolov71/runs/detect/exp5/"
    event_handler = MyHandler(model)
    observer = Observer()
    observer.schedule(event_handler, path, recursive=True)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()